In [59]:
import sys
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import os

from sklearn.model_selection import train_test_split

import json

# Initial global var

In [60]:
## 미리 Global 변수를 지정하자. 파일 명, 파일 위치, 디렉토리 등이 있다.

DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_Q1_DATA_FILE = 'train_q1.npy'
TRAIN_Q2_DATA_FILE = 'train_q2.npy'
TRAIN_LABEL_DATA_FILE = 'train_label.npy'
NB_WORDS_DATA_FILE = 'nb_words.json'

## 학습에 필요한 파라메터들에 대해서 지정하는 부분이다.

BATCH_SIZE = 128
EPOCH = 50
HIDDEN = 64
BUFFER_SIZE = 2048

TEST_SPLIT = 0.1
RNG_SEED = 13371447
EMBEDDING_DIM = 128
MAX_SEQ_LEN = 25

# Load Dataset

In [61]:
## 데이터를 불러오는 부분이다. 효과적인 데이터 불러오기를 위해, 미리 넘파이 형태로 저장시킨 데이터를 로드한다.

q1_data = np.load(open(DATA_IN_PATH + TRAIN_Q1_DATA_FILE, 'rb'))
q2_data = np.load(open(DATA_IN_PATH + TRAIN_Q2_DATA_FILE, 'rb'))
labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))
prepro_configs = None

with open(DATA_IN_PATH + NB_WORDS_DATA_FILE, 'r') as f:
    prepro_configs = json.load(f)

In [62]:
VOCAB_SIZE = prepro_configs['vocab_size']

# Split train and test dataset

In [63]:
q1_data_len = np.array([min(len(x), MAX_SEQ_LEN) for x in q1_data])
q2_data_len = np.array([min(len(x), MAX_SEQ_LEN) for x in q2_data])

In [64]:
## 데이터를 나누어 저장하자. sklearn의 train_test_split을 사용하면 유용하다. 하지만, 쿼라 데이터의 경우는
## 입력이 1개가 아니라 2개이다. 따라서, np.stack을 사용하여 두개를 하나로 쌓은다음 활용하여 분류한다.

X = np.stack((q1_data, q2_data), axis=1)
y = labels
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RNG_SEED)

train_Q1 = train_X[:,0]
train_Q2 = train_X[:,1]
test_Q1 = test_X[:,0]
test_Q2 = test_X[:,1]

In [65]:
def rearrange(base, hypothesis, labels):
    features = {"base": base, "hypothesis": hypothesis}
    return features, labels

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((train_Q1, train_Q2, train_y))
    dataset = dataset.shuffle(buffer_size=len(train_Q1))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(rearrange)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((test_Q1, test_Q2, test_y))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(rearrange)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [73]:
def Malstm(features, labels, mode, params):
        
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    labels = tf.to_float(tf.reshape(labels, [-1,1]))

    def basic_bilstm_network(inputs, name, reuse=tf.AUTO_REUSE):
        with tf.variable_scope(name, reuse=reuse):
            lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(num_units = HIDDEN, activation = tf.nn.tanh)
            lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(num_units = HIDDEN, activation = tf.nn.tanh)
            outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw = lstm_fw_cell,
                                                              cell_bw = lstm_bw_cell,
                                                              inputs = inputs,
                                                              dtype = tf.float32)
            concat = tf.concat([outputs[0], outputs[1]], axis=2)
            final_state = tf.keras.layers.Dense(128)(concat[:,-1,:])

        return final_state
        
    embedding = tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
    
    base_embedded_matrix = embedding(features['base'])
    hypothesis_embedded_matrix = embedding(features['hypothesis'])

    base_embedded_matrix = tf.keras.layers.Dropout(0.2)(base_embedded_matrix)
    hypothesis_embedded_matrix = tf.keras.layers.Dropout(0.2)(hypothesis_embedded_matrix)    
    
    base_sementic_matrix = basic_bilstm_network(base_embedded_matrix, 'base')
    hypothesis_sementic_matrix = basic_bilstm_network(hypothesis_embedded_matrix, 'hypothesis')
    
#     merged_matrix = tf.concat([base_sementic_matrix, hypothesis_sementic_matrix], -1)

    logit_layer = K.exp(-K.sum(K.abs(base_sementic_matrix -  hypothesis_sementic_matrix), axis=1, keepdims=True))
                
    if PREDICT:
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  predictions={
                      'prob':tf.nn.sigmoid(logit_layer)
                  })
    
    loss =tf.losses.sigmoid_cross_entropy(labels, logit_layer)
    
    if EVAL:
        accuracy = tf.metrics.accuracy(labels, tf.nn.sigmoid(logit_layer))
        eval_metric_ops = {'acc': accuracy}
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  eval_metric_ops= eval_metric_ops,
                  loss=loss)

    elif TRAIN:
        global_step = tf.train.get_global_step()
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  train_op=train_op,
                  loss=loss)

    
params = {'embed_init': tf.random_uniform_initializer(-1.0, 1.0)}

In [76]:
model_dir = os.path.join(os.getcwd(), DATA_OUT_PATH + "/checkpoint/")
os.makedirs(model_dir, exist_ok=True)

config_tf = tf.estimator.RunConfig()
config_tf._save_checkpoints_steps = 100
config_tf._save_checkpoints_secs = None
config_tf._keep_checkpoint_max =  2
config_tf._log_step_count_steps = 5

dssm_est = tf.estimator.Estimator(Malstm, model_dir=DATA_OUT_PATH + 'checkpoint', config=config_tf, params=params)
dssm_est.train(train_input_fn, steps=10000)

INFO:tensorflow:Using config: {'_model_dir': './data_out/checkpoint', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 5, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x21278ccf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/checkpoint/model.ckpt-11
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 11 i

INFO:tensorflow:loss = 0.6931473, step = 367 (0.968 sec)
INFO:tensorflow:global_step/sec: 5.54431
INFO:tensorflow:loss = 0.6931473, step = 372 (0.902 sec)
INFO:tensorflow:global_step/sec: 5.13704
INFO:tensorflow:loss = 0.6931473, step = 377 (0.974 sec)
INFO:tensorflow:global_step/sec: 5.96956
INFO:tensorflow:loss = 0.6931473, step = 382 (0.837 sec)
INFO:tensorflow:global_step/sec: 5.6133
INFO:tensorflow:loss = 0.6931473, step = 387 (0.891 sec)
INFO:tensorflow:global_step/sec: 5.94129
INFO:tensorflow:loss = 0.6931473, step = 392 (0.842 sec)
INFO:tensorflow:global_step/sec: 6.06086
INFO:tensorflow:loss = 0.6931473, step = 397 (0.825 sec)
INFO:tensorflow:global_step/sec: 5.91909
INFO:tensorflow:loss = 0.6931473, step = 402 (0.845 sec)
INFO:tensorflow:global_step/sec: 5.74108
INFO:tensorflow:loss = 0.6931473, step = 407 (0.870 sec)
INFO:tensorflow:Saving checkpoints for 411 into ./data_out/checkpoint/model.ckpt.
INFO:tensorflow:global_step/sec: 1.75367
INFO:tensorflow:loss = 0.6931473, ste

KeyboardInterrupt: 

In [ ]:
TEST_Q1_DATA_FILE = 'test_q1.npy'
TEST_Q2_DATA_FILE = 'test_q2.npy'

test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))

predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"q":test_q1_data, 
                                                         "sim_q":test_q2_data}, 
                                                      shuffle=False)

predictions = np.array([p['prob'] for p in est.predict(input_fn=predict_input_fn)])

In [ ]:
import pandas as pd

DEFAULT_PATH='~/.kaggle/competitions/quora-question-pairs/'

test = pd.read_csv(DEFAULT_PATH + "test.csv", encoding='utf-8')
test = test.dropna()

In [ ]:
output = pd.DataFrame(data={"test_id":test["test_id"], "is_duplicate": list(predictions)} )
output.to_csv(DATA_OUT_PATH + "rnn_predict.csv", index=False, quoting=3 )

In [ ]:
time_start = datetime.now()

# validation_monitor = tf.contrib

print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................") 

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

# train_spec = tf.estimator.TrainSpec(train_input_fn)
# eval_spec = tf.estimator.EvalSpec(test_input_fn)

dssm_est = tf.estimator.Estimator(Malstm, model_dir=model_dir, config=config_tf, params=params)
dssm_est.train(train_input_fn, steps=EPOCH)

# tf.estimator.train_and_evaluate(dssm_est, train_spec, eval_spec)

# For prediction or extract values
# prediction = est.predict(eval_input_fn)

# for i, p in enumerate(prediction):
#     print(i, p['sim_q_sem'])

time_end = datetime.now()

print(".......................................")
print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))